In [1]:
from bs4 import BeautifulSoup
import csv
import requests
import numpy as np
import copy
import urllib.request

In [2]:
'''
#METHOD 1
url="http://blabla.com"
soup = bs(urllib.urlopen(url))

#METHOD 2
page = requests.get("https://afltables.com/afl/seas/1901.html")
page
soup = BeautifulSoup (page.content, 'html5lib')
'''

'\n#METHOD 1\nurl="http://blabla.com"\nsoup = bs(urllib.urlopen(url))\n\n#METHOD 2\npage = requests.get("https://afltables.com/afl/seas/1901.html")\npage\nsoup = BeautifulSoup (page.content, \'html5lib\')\n'

In [3]:
def game_list(souploc):
    data4 = []
    table = souploc.findAll('table')
    for tb in range(len(table)):
        data = []
        table_body = table[tb].find('tbody')
        if(str(table_body)[:40] == "<tbody><tr><td valign=\"top\" width=\"85%\">"):
            rows = table_body.findAll('tr')
            for row in rows:
                cols = row.find_all('td')
                cols = [ele.text.strip() for ele in cols]
                data.append([ele for ele in cols if ele]) # Get rid of empty values
                
                if data[0]:
                    data[0].pop(0)
                else:
                    continue
                data2 = []
                data3 = []
                counter  = 0 
                for point in range(len(data[0])):
                    if(point % 2 == 0 ):
                        counter += 1
                        if(data[0][point][:2]=="Rd"):
                            break
                        data2.append(data[0][point])
                        if(counter == 4):
                            if(not data2[1].isnumeric() or not data2[3].isnumeric()):
                                break
                            data3.extend([data2[:]])
                            data2.clear()
                            counter = 0
                            
                if not data3:
                    continue
                else:
                    data4.extend(data3[:])
                    data.clear()
                    data2.clear()
                    data3.clear()
    return data4

In [4]:
urltest = ["https://afltables.com/afl/seas/2020.html"]
urllist = ["https://afltables.com/afl/seas/1999.html","https://afltables.com/afl/seas/2000.html",
           "https://afltables.com/afl/seas/2001.html","https://afltables.com/afl/seas/2002.html",
           "https://afltables.com/afl/seas/2003.html","https://afltables.com/afl/seas/2004.html",
           "https://afltables.com/afl/seas/2005.html","https://afltables.com/afl/seas/2006.html",
           "https://afltables.com/afl/seas/2007.html","https://afltables.com/afl/seas/2008.html",
           "https://afltables.com/afl/seas/2009.html","https://afltables.com/afl/seas/2010.html",
           "https://afltables.com/afl/seas/2011.html","https://afltables.com/afl/seas/2012.html",
           "https://afltables.com/afl/seas/2013.html","https://afltables.com/afl/seas/2014.html",
           "https://afltables.com/afl/seas/2015.html","https://afltables.com/afl/seas/2016.html",
           "https://afltables.com/afl/seas/2017.html","https://afltables.com/afl/seas/2018.html",
           "https://afltables.com/afl/seas/2019.html","https://afltables.com/afl/seas/2020.html"]

In [5]:
def list_maker(url):
    soup = BeautifulSoup(urllib.request.urlopen(url), 'html5lib')
    gamedata = game_list(soup)
    links = soup.findAll('a',{'href': True})
    linkdata = []
    
    for link in links:
        if(link['href'][:15]=="../stats/games/"):
            stredit = "https://afltables.com/afl"+str(link['href'][2:])
            linkdata.append(stredit)
    
    for game in range(len(gamedata)):
        #print(linkdata[game])
        gamedata[game].insert(4,linkdata[game])
        #print(gamedata[game])  
        
    return gamedata

In [6]:
full_list = []
full_list.clear()
for url in urllist:
    full_list.extend(list_maker(url))

In [7]:
with open('game_list.csv', 'w', newline='') as myfile:
    wr = csv.writer(myfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    for row in full_list:
        wr.writerow(row)